In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import time
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix, average_precision_score, f1_score, roc_curve, auc
from sklearn.metrics import precision_recall_curve
import seaborn as sns; sns.set()
sns.set_style('whitegrid')
sns.set_palette('Set2')
%matplotlib inline
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

# Define Helper Functions

In [ ]:
def probability_to_label(probabilities, threshold=0.5):
    probabilities = list(probabilities)
    th = threshold
    predictions = [1 if i > th else 0 for i in probabilities]
    return predictions

In [ ]:
def plot_cm(y_true, y_pred,
                          title='Confusion Matrix',
                          cmap=plt.cm.Blues):
    sns.set_style('white')
    
    cm = confusion_matrix(y_test, y_pred)
    cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis] # normarlize 
    
    from sklearn.utils.multiclass import unique_labels
    classes = unique_labels(y_true, y_pred)
    
    fig, ax = plt.subplots()
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)
    # We want to show all ticks...
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           # ... and label them with the respective list entries
           xticklabels=classes, yticklabels=classes,
           title=title,
           ylabel='True label',
           xlabel='Predicted label')

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")
    # Loop over data dimensions and create text annotations.
    fmt = '.2f' 
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    fig.tight_layout()
    
    pass

In [ ]:
def Imp_of_fea(name,weight):
    import operator
    import collections
    dic = dict(zip(name,weight))
    key_drop = [k for k in dic.keys() if  k == 'icustay_age_group']
    for k in key_drop:
        del dic[k]
    dic = sorted(dic.items(), key=operator.itemgetter(1))
    sorted_dict = collections.OrderedDict(dic)
    plt.style.use('seaborn-darkgrid')
    plt.figure(figsize=(5,20))
    plt.barh(list(sorted_dict.keys()),list(sorted_dict.values()))
    plt.title('Importance of features in 30 day mortality')

In [ ]:
def plot_ROC(fpr, tpr, auc, title = 'ROC curve'):
    sns.set_style('whitegrid')
    plt.plot([0, 1], [0, 1], 'k--')
    plt.plot(fpr, tpr, label=' (AUC = {:.3f})'.format(auc))
    plt.xlabel('False positive rate')
    plt.ylabel('True positive rate')
    plt.title(title)
    plt.legend(loc='best')
    plt.show()
    pass

In [ ]:
def plot_PRC(precision, recall, ap, title= 'Precision-Recall Curve '):
    sns.set_style('whitegrid')

    plt.figure
#     plot(result5_1[0], result5_1[1],linestyle = "-", color = 'C0')#,label = 'AUC = %0.2f' % result5_1[3])
    plt.plot(recall, precision, lw=2, label='AP = %0.4f' % ap )
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.title(title)
    plt.legend(loc="best")
#     plt.show()
    return 

In [ ]:
def get_weights(y):
    zeros = y[y==0].shape[0]
    ones = y[y==1].shape[0]
    weight = np.array([zeros, ones])*1.0/y.shape[0]
    y = np.where(y==0,weight[1],y)
    y = np.where(y==1,weight[0],y)
    return y

## Load Data

In [ ]:
%run LoadData1Y.ipynb

In [ ]:
X_train.head()

In [ ]:
y_train.head()

# Xgboost Decision Tree

### Apply SearchGrid Cross Validation

In [ ]:
param_grid = { 
    'learning_rate' : list(np.linspace(0.1,0.5,5)),
    'n_estimators'  : list(np.logspace(1,3,3).astype(int))
}
model = GridSearchCV(
    estimator = XGBClassifier(verbosity=2),
    param_grid = param_grid,
    n_jobs = -1,
    verbose = 3,
    scoring = 'accuracy',
    cv = 5
).fit(X_train,y_train,get_weights(y_train.values))

In [ ]:
best_model = model.best_estimator_
ypred = best_model.predict_proba(X_test)
np.savetxt("plot metric/y_score_1y_xgboost.csv", ypred[:,1], delimiter=",")

### Accuracy Score

In [ ]:
best_model.score(X_test,y_test)

### Importance of Features

In [ ]:
%run ImportPlotFunctions.ipynb
name = X_train.columns
weight = best_model.feature_importances_
feature_importance_bar(name,weight,drop_botn = 45)

### Confusion Matrix

In [ ]:
ypred_label =  probability_to_label(ypred[:,1], threshold=0.5)
plot_cm(y_test, ypred_label)

### ROC Curve and AUC

In [ ]:
fpr, tpr, thresholds = roc_curve(y_test, ypred[:,1])
yauc = auc(fpr,tpr)
plot_ROC(fpr, tpr, yauc)

### PRC Curve

In [ ]:
avg_prec = average_precision_score(y_test, ypred_label)
prec, recall,_ = precision_recall_curve(y_test,ypred[:,1])
plot_PRC(prec, recall, avg_prec, title= 'Precision-Recall Curve ')

### F1 Score

In [ ]:
f1_score(y_test,ypred_label)